In [46]:
# Libraries
import tensorflow as tf

# Making tf dataset from simple python list

In [47]:
# From list to tf dataset
daily_sales_numbers = [21, 22, -108, 31, -1, 32, 34, 31]
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [48]:
# For looking into the dataset
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
-108
31
-1
32
34
31


In [49]:
# For looking into 1st n values in the dataset
for sales in tf_dataset.take(3):
    print(sales.numpy())

21
22
-108


In [50]:
# Filtering the dataset
tf_filtered_dataset = tf_dataset.filter(lambda x: x>0)
for sales in tf_filtered_dataset.as_numpy_iterator():
    print(sales)

21
22
31
32
34
31


In [51]:
# Mapping the dataset
tf_mapped_dataset = tf_filtered_dataset.map(lambda x: x*72)
for sales in tf_mapped_dataset.as_numpy_iterator():
    print(sales)

1512
1584
2232
2304
2448
2232


In [52]:
# Randomizing the dataset
tf_dataset = tf_mapped_dataset.shuffle(3)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

2232
1584
2448
2304
1512
2232


In [53]:
# Batching the training sample
for sales in tf_dataset.batch(4).as_numpy_iterator():
    print(sales)

[1512 1584 2304 2232]
[2232 2448]


In [54]:
# Doing all the above in single line
daily_sales_numbers = [21, 22, -108, 31, -1, 32, 34, 31]
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)

tf_1line_dataset = tf_dataset.filter(lambda x: x>0).map(lambda y: y*72).shuffle(2).batch(2)

for sales in tf_1line_dataset.as_numpy_iterator():
    print(sales)

[1512 1584]
[2304 2448]
[2232 2232]


# Doing This For Images

In [55]:
images_dataset = tf.data.Dataset.list_files("images/*/*", shuffle=True)
for file in images_dataset.take(3):
    print(file.numpy())

b'images\\cat\\7 Foods Your Cat Can_t Eat.jpg'
b'images\\dog\\The 20 Best Dog Breeds for Runners....jpg'
b'images\\dog\\Calculate Your Dog_s Age With This New....jpg'


In [56]:
class_names = ["cat", "dog"]
image_count = len(images_dataset)
image_count

130

In [57]:
train_size = int(image_count*0.8)
train_ds = images_dataset.take(train_size)  # It will take 1st 80% as raining dataset
test_ds = images_dataset.skip(train_size)   # It will skip he 1st 80% samples

In [58]:
len(train_ds), len(test_ds)

(104, 26)

# Taking Labels from the images path

In [59]:
import os
def get_label(file_path):
    return tf.strings.split(file_path, os.path.sep)[-2]

for label in train_ds.map(get_label).take(3):
    print(label)

tf.Tensor(b'dog', shape=(), dtype=string)
tf.Tensor(b'dog', shape=(), dtype=string)
tf.Tensor(b'dog', shape=(), dtype=string)


In [60]:
def process_image(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [128, 128])
    return img, label

train_ds = train_ds.map(process_image)
for img, label in train_ds.take(3):
    print(img.numpy()[0][0])
    print(label.numpy())

[183.34375 190.34375  94.34375]
b'dog'
[ 7.6796875 25.679688  11.919922 ]
b'dog'
[175.41406 176.41406 171.41406]
b'cat'


# Scaling

In [61]:
def scale(image, label):
    return image/255, label

for image, label in train_ds.map(scale).take(3):
    print(image.numpy()[0][0])
    print(label.numpy())

[1. 1. 1. 0.]
b'dog'
[0.60398287 0.6628064  0.6510417 ]
b'dog'
[0.42360982 0.49419808 0.5412569 ]
b'dog'
